---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [69]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [97]:
month_dict = {"jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6, "jul": 7, "aug": 8, "sep": 9, "oct": 10, "nov": 11, "dec": 12}
def month_sub(month):
    month_ab = month[:3].lower()
    if month_ab in month_dict:
        return month_dict[month_ab]
    return -1
    

In [129]:
import re
def day_parse(x):
    pattern1 = "(?P<month>[\d]{1,2})[\/\-](?P<day>[\d]{1,2})[\/\-](?P<year>[\d]{2,4})"#04/20/2009; 04/20/09; 4/20/09; 4/3/09
    pattern2 = "(?P<month>[A-Z][a-z]+)[\s\/.\-]+(?P<day>[\d]{1,2})[\-\s,]+(?P<year>[\d]{2,4})" #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar 20 2009; Mar. 20, 2009; 
    pattern3 = "(?P<day>[\d]{1,2})[\-\s]+(?P<month>[A-Z][a-z]+)[\-\s\/.,]+(?P<year>[\d]{2,4})" #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    pattern4 = "(?P<month>[A-Z][a-z]+)[\.\-\s]+(?P<day>[\d]{1,2})[a-z]{2}[\-\s,]+(?P<year>[\d]{2,4})" #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    pattern5 = "(?P<day>[\d]{1,2})[a-z]{2}[\-\s,]+(?P<month>[A-Z][a-z]+)[.\-\s]+(?P<year>[\d]{2,4})"
    
    day, month, year = -1, -1, -1
    
    list1 = re.findall(pattern1, x)
    if len(list1) > 0:
        dates = list1[0]
        day, month, year = dates[1], dates[0], dates[2]
    
    list2 = re.findall(pattern2, x)
    if len(list2) > 0:
        for dates in list2:
            day, month, year = dates[1], month_sub(dates[0]), dates[2]
            if month != -1:
                break
    
    list3 = re.findall(pattern3, x)
    if len(list3) > 0:
        for dates in list3:
            day, month, year = dates[0], month_sub(dates[1]), dates[2]
            if month != -1:
                break
    
    list4 = re.findall(pattern4, x)
    if len(list4) > 0:
        for dates in list4:
            day, month, year = dates[1], month_sub(dates[0]), dates[2]
            if month != -1:
                break
    
    list5 = re.findall(pattern5, x)
    if len(list5) > 0:
        for dates in list5:
            day, month, year = dates[0], month_sub(dates[1]), dates[2]
            if month != -1:
                break
    
    if month != -1:
        if len(year) == 2:
            return(int(day), int(month), int("19"+year))
        else:
            return(int(day), int(month), int(year))
    
    return (-1, -1, -1)


In [128]:
def month_parse(x):
    
    pattern1 = "(?P<month>[A-Z][a-z]+)[\s\/.,]+(?P<year>[\d]{2,4})"
    pattern2 = "(?P<month>[\d]{1,2})[\/](?P<year>[\d]{2,4})"
    
    day, month, year = -1, -1, -1
    
    list1 = re.findall(pattern1, x)
    if len(list1) > 0:
        for dates in list1:
            day, month, year = "1", month_sub(dates[0]), dates[1]
            if month != -1:
                break
    
    list2 = re.findall(pattern2, x)
    if len(list2) > 0:
        dates = list2[0]
        day, month, year = "1", dates[0], dates[1]
        
    if month != -1:
        if len(year) == 2:
            return(int(day), int(month), int("19"+year))
        else:
            return(int(day), int(month), int(year))    
        
    return (-1, -1, -1)   


In [167]:
def year_parse(x):
    
    pattern1 = "(?P<year>[\d]{4})"
    pattern2 = "(?P<year>[\d]{2})"
    day, month, year = -1, -1, -1
    list1 = re.findall(pattern1, x)
    if len(list1) > 0:
        dates = list1[0]
        day, month, year = "1", "1", dates
        return(int(day), int(month), int(year))  
    
    list2 = re.findall(pattern2, x)
    
    if len(list2) > 0:
        dates = list2[0]
        day, month, year = "1", "1", dates
        return(int(day), int(month), int("19"+year))
        
    return (day, month, year) 


In [73]:
def none_parse(x):
    return (0, 0, 0)

In [169]:
import re
import datetime
def date_sorter():
    # Your code here
    num_rows = df.shape[0]
    return_list = []
    func_list = [day_parse, month_parse, year_parse, none_parse]
    
    for i in range(num_rows):
        x = df[i]
        for func in func_list:
            date = func(x)           
            if date != (-1, -1, -1):
                return_list.append((i, date))
                break
    return_list = sorted(return_list, key = lambda x: (x[1][2], x[1][1], x[1][0]))
    
    return_idx = [x[0] for x in return_list]
    
    return pd.Series(return_idx)
                
        

print(date_sorter())

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64


In [117]:
print(df[248])

;Trazodone 50-100 mg QHS, had also been on this long-time when it was d/ced in July 1995.



In [103]:
x = "In 1980, patient was living in Naples and described being upset by the amount of noise generated by the neighbor living under his apartment.  He noted the woman was a heroin user and when she refused to quiet down, he walked into her apartment and trashed it.  He was arrested for breaking and entering and destruction of property and was placed on probation."
month_parse(x)

(1, -1, 1980)

In [127]:
month_parse(df[248])

(1, 50, 100)

In [174]:
month_parse(df[231])

(1, 5, 2016)